## **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import glob 
import keras
import seaborn as sns
import pickle
import sklearn

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.3 MB/s 


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
from sklearn.model_selection import RandomizedSearchCV
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Preprocessing**

In [ ]:
raw_csv_data=pd.read_csv("/content/drive/MyDrive/datasetW4/Final_Dataset_withoutStatus.csv")
df=raw_csv_data.copy()
pd.options.display.max_columns= None
pd.options.display.max_rows= 10
df

,consignment_id,consignment_note,Account_name,account_type,AssignedTo,PriorityLevel,ConCreated,InActive,SenderCity,ReceiverCity,SenderSuburb,ReceiverSuburb,ServiceType,Carrier_name,leg_type,from_leg,to_leg,depotname,LeggingCost
0,13086FE7-12A1-4467-964C-0000047EE392,IFL3022016,BELGOTEX NZ LTD,Customer,NaN,NaN,2022-05-14,False,Auckland,Te Awamutu,MANGERE,NaN,BOP - Waikato,STL LINEHAUL,PU/LH/DL,AUCKLAND,TE AWAMUTU,STL AUCKLAND DEPOT,0.00
1,0AA9C307-3746-4302-A0B6-0004F0ACD64A,ECC0040335,AUSLINK LOGISTICS LIMITED,Customer,NaN,NaN,2022-01-19,False,Christchurch,Christchurch,Wigram,Avondale,Metro General,LK TRANSPORT Store Rite Logistics,PU/DL,Christchurch,Christchurch,NaN,13.40
2,E6A42B8A-B673-4B7B-8DFF-0005F5264B82,IFL3022422,BELGOTEX NZ LTD,Customer,NaN,NaN,2022-03-08,False,Auckland,Wellington,Wiri,Pipitea,SuperEconomy,STL LINEHAUL,PU/LH/DL,AUCKLAND,WELLINGTON,STL Wellington,47.16
3,45559152-2187-4C29-9EF5-0008C207BABF,ECC0039329,DHL EXPRESS - CHCH,Customer,NaN,NaN,2021-11-22,False,Christchurch,Lake Hayes,Avonhead,Lake Hayes Estate,Economy,bOOTH'S TRANSPORT (SI) LTD,PU/LH/DL,Christchurch,Queenstown,NaN,30.00
4,9A405A44-F95C-40D9-8351-000B34C4AD93,IFL3021888,BELGOTEX NZ LTD,Customer,NaN,NaN,2022-05-14,False,Auckland,Wellington,MANGERE,Hutt Central,SuperEconomy,STL LINEHAUL,PU/LH/DL,AUCKLAND,WELLINGTON,STL AUCKLAND DEPOT,47.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23643,19E47EEC-EEE7-452E-8A8A-FFF58C7BD36B,IFL9024928,BELGOTEX NZ LTD,Customer,NaN,NaN,2022-06-06,False,Auckland,AUCKLAND,MANGERE south,PUKEKOHE,SuperEconomy,CARGO PLUS LTD,PU/LH/DL,AUCKLAND,PUKEKOHE,NaN,34.77
23644,E83DCC0F-E207-42E2-9A34-FFFA6BF0399F,ECC0041208,TAI PING TRADING COMPANY LTD,Customer,NaN,NaN,2022-01-25,False,Auckland,Timaru,Penrose,NaN,SuperEconomy,STL LINEHAUL,PU/LH,Auckland,Christchurch,STL AUCKLAND DEPOT,51.91
23645,E83DCC0F-E207-42E2-9A34-FFFA6BF0399F,ECC0041208,TAI PING TRADING COMPANY LTD,Customer,NaN,NaN,2022-01-25,False,Auckland,Timaru,Penrose,NaN,SuperEconomy,bOOTH'S TRANSPORT (SI) LTD,LH/DL,Christchurch,Timaru,NaN,32.39
23646,8F8F2508-9E79-4875-9215-FFFB36CEC073,ECC0046767,PYROTEK PRODUCTS LTD,Customer,HAYLEYW,overdue,2022-09-13,False,Auckland,Whangarei,East Tamaki,Port Whangarei,Economy,FREIGHTPLUS LTD,PU/LH/DL,AUCKLAND,WHANGAREI,NaN,35.35


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23648 entries, 0 to 23647
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   consignment_id    23648 non-null  object 
 1   consignment_note  23648 non-null  object 
 2   Account_name      23648 non-null  object 
 3   account_type      23648 non-null  object 
 4   AssignedTo        637 non-null    object 
 5   PriorityLevel     626 non-null    object 
 6   ConCreated        23648 non-null  object 
 7   InActive          23648 non-null  bool   
 8   SenderCity        23644 non-null  object 
 9   ReceiverCity      23634 non-null  object 
 10  SenderSuburb      22001 non-null  object 
 11  ReceiverSuburb    17034 non-null  object 
 12  ServiceType       23648 non-null  object 
 13  Carrier_name      23648 non-null  object 
 14  leg_type          23648 non-null  object 
 15  from_leg          23648 non-null  object 
 16  to_leg            23648 non-null  object

In [ ]:
pd.set_option('display.max_rows', None)
df['Carrier_name'].value_counts()

STL LINEHAUL                          9082
CARGO PLUS LTD                        3951
bOOTH'S TRANSPORT (SI) LTD            2253
BASCIK TRANSPORT                      2165
FREIGHTPLUS LTD                       1751
MAINSTREAM NZ LTD                     1692
T&G TRANSPORT                          842
LK TRANSPORT Store Rite Logistics      666
HALLS REFRIGERATION                    185
TRANSFREIGHT                           164
LINFOX                                 138
ANGEL TRANSPORT                        117
PASS THE PARCEL                         74
COMBINED FREIGHTLINES                   63
URGENT COURIERS LTD                     57
QANTAS COURIERS                         46
METRO URGENT                            41
MACKENZIE TRANSPORT LTD                 40
JETS TRANSPORT LTD                      38
SUB60                                   26
SEALINK LOGISTICS                       23
SOLLYS                                  22
ADEPT LOGISTICS LTD                     22
TRANSCON   

In [ ]:
df['Account_name'].value_counts()

In [ ]:
df['SenderCity'].value_counts()

In [ ]:
df['ServiceType'].value_counts()

In [ ]:
df.describe()

,LeggingCost
count,23646.000000
mean,126.041854
std,327.905371
min,0.000000
25%,20.380000
50%,48.570000
75%,102.455000
max,18200.000000


In [ ]:
Ldata = df.copy()

In [ ]:
Ldata=pd.concat([Ldata['Account_name'], Ldata['SenderCity'], Ldata['ReceiverCity'], Ldata['ServiceType'], Ldata['from_leg'], Ldata['to_leg'], Ldata['leg_type'], Ldata['LeggingCost'], Ldata['Carrier_name'] ], axis=1)
Ldata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
0,BELGOTEX NZ LTD,Auckland,Te Awamutu,BOP - Waikato,AUCKLAND,TE AWAMUTU,PU/LH/DL,0.00,STL LINEHAUL
1,AUSLINK LOGISTICS LIMITED,Christchurch,Christchurch,Metro General,Christchurch,Christchurch,PU/DL,13.40,LK TRANSPORT Store Rite Logistics
2,BELGOTEX NZ LTD,Auckland,Wellington,SuperEconomy,AUCKLAND,WELLINGTON,PU/LH/DL,47.16,STL LINEHAUL
3,DHL EXPRESS - CHCH,Christchurch,Lake Hayes,Economy,Christchurch,Queenstown,PU/LH/DL,30.00,bOOTH'S TRANSPORT (SI) LTD
4,BELGOTEX NZ LTD,Auckland,Wellington,SuperEconomy,AUCKLAND,WELLINGTON,PU/LH/DL,47.16,STL LINEHAUL
5,FORCE4 LIMITED,Wellington,SILVERDALE,Metro General,WELLINGTON,AUCKLAND,PU/DL,0.00,GREAT BARRIER CARTAGE LTD
6,BELGOTEX NZ LTD,Auckland,New Plymouth,SuperEconomy,AUCKLAND,NEW PLYMOUTH,PU/LH/DL,48.57,STL LINEHAUL
7,BELGOTEX NZ LTD,Auckland,Auckland,Metro General,AUCKLAND,GREENLANE,PU/LH/DL,19.56,CARGO PLUS LTD
8,BELGOTEX NZ LTD,Auckland,AUCKLAND,Metro General,AUCKLAND,AUCKLAND,PU/LH/DL,18.90,CARGO PLUS LTD
9,BELGOTEX NZ LTD,Auckland,New Plymouth,SuperEconomy,AUCKLAND,NEW PLYMOUTH,PU/LH/DL,48.57,STL LINEHAUL


In [ ]:
Ldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23648 entries, 0 to 23647
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  23648 non-null  object 
 1   SenderCity    23644 non-null  object 
 2   ReceiverCity  23634 non-null  object 
 3   ServiceType   23648 non-null  object 
 4   from_leg      23648 non-null  object 
 5   to_leg        23648 non-null  object 
 6   leg_type      23648 non-null  object 
 7   LeggingCost   23646 non-null  float64
 8   Carrier_name  23648 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.6+ MB


In [ ]:
Ldata['ReceiverCity'].isnull().sum()

14

In [ ]:
Ldata.drop(Ldata.loc[Ldata['LeggingCost'].isnull()].index, inplace = True)
Ldata.drop(Ldata.loc[Ldata['SenderCity'].isnull()].index, inplace = True)
Ldata.drop(Ldata.loc[Ldata['ReceiverCity'].isnull()].index, inplace = True)

In [ ]:
Ldata.drop(Ldata.loc[Ldata['LeggingCost'] ==0.0].index, inplace = True)

In [ ]:
Ldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19899 entries, 1 to 23647
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  19899 non-null  object 
 1   SenderCity    19899 non-null  object 
 2   ReceiverCity  19899 non-null  object 
 3   ServiceType   19899 non-null  object 
 4   from_leg      19899 non-null  object 
 5   to_leg        19899 non-null  object 
 6   leg_type      19899 non-null  object 
 7   LeggingCost   19899 non-null  float64
 8   Carrier_name  19899 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.5+ MB


###**Mapping categorical to numerical**

In [ ]:
Ldata['SenderCity'] = Ldata['SenderCity'].str.lower()
Ldata['ReceiverCity'] = Ldata['ReceiverCity'].str.lower()
Ldata['Carrier_name'] = Ldata['Carrier_name'].str.lower()
Ldata['Account_name'] = Ldata['Account_name'].str.lower()
Ldata['ServiceType'] = Ldata['ServiceType'].str.lower()
Ldata['from_leg'] = Ldata['from_leg'].str.lower()
Ldata['to_leg'] = Ldata['to_leg'].str.lower()
Ldata['leg_type'] = Ldata['leg_type'].str.lower()
Ldata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,christchurch,christchurch,metro general,christchurch,christchurch,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,auckland,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,christchurch,lake hayes,economy,christchurch,queenstown,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,auckland,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,auckland,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,auckland,auckland,metro general,auckland,greenlane,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,auckland,auckland,metro general,auckland,auckland,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,auckland,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,auckland,whangarei,economy,auckland,whangarei,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,tauranga,rotorua,bop - waikato,auckland,rotorua,pu/lh/dl,20.60,stl linehaul


####**CITY**

In [ ]:
import csv

file = open("/content/drive/MyDrive/datasetW4/Cities.csv", "r")
data = list(csv.reader(file, delimiter=","))
file.close()

In [ ]:
citylist= list()
for i in data:
  for j in i:
    #print(j)
    citylist.append(str(j).lower())

In [ ]:
c=1
cityDict = {}
cityDict = {citylist[i]: int(i+1) for i in range(len(citylist))}
cityDict

{'hastings': 1,
 'cable bay': 2,
 'baylys beach': 3,
 'aniseed valley': 4,
 'otanomomo': 5,
 'tirau': 6,
 'eltham': 7,
 'te teko': 8,
 'lake hayes': 9,
 'mangatawhiri': 10,
 'matapouri': 11,
 'tekapo': 12,
 'balcairn': 13,
 'ōpōtiki': 14,
 'oamaru': 15,
 'opotiki': 16,
 'paihia': 17,
 'bunnythorpe': 18,
 'franz josef': 19,
 'pleasant valley': 20,
 'kaingaroa forest': 21,
 'levels': 22,
 'brooklyn': 23,
 'teviotdale': 24,
 'silverdale': 25,
 'kawakawa': 26,
 'ashhurst': 27,
 'eskdale': 28,
 'haumoana': 29,
 'southbridge': 30,
 'hikurangi': 31,
 'waihi': 32,
 'albert town': 33,
 'bannockburn': 34,
 'wellsford': 35,
 'murupara': 36,
 'reefton': 37,
 'morven': 38,
 'pokeno': 39,
 'pōkeno': 40,
 'waimauku': 41,
 'aokautere': 42,
 'whanagrei heads': 43,
 'montalto': 44,
 'pipiwai': 45,
 'fairview': 46,
 'west eyreton': 47,
 'little wanganui': 48,
 'auckland': 49,
 'whangarei': 50,
 'whangārei': 51,
 'waipukurau': 52,
 'branxholme': 53,
 'orari': 54,
 'pohuehue': 55,
 'hapuku': 56,
 'edendale

In [ ]:
with open('cityDict.pkl', 'wb') as f:
    pickle.dump(cityDict, f)
        
with open('cityDict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
Legdata = Ldata.copy()

In [ ]:
Legdata['SenderCity']=Legdata['SenderCity'].map(cityDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,424.0,christchurch,metro general,christchurch,christchurch,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,49.0,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,424.0,lake hayes,economy,christchurch,queenstown,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,49.0,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,49.0,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,49.0,auckland,metro general,auckland,greenlane,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,49.0,auckland,metro general,auckland,auckland,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,49.0,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,49.0,whangarei,economy,auckland,whangarei,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,191.0,rotorua,bop - waikato,auckland,rotorua,pu/lh/dl,20.60,stl linehaul


In [ ]:
Legdata['ReceiverCity']=Legdata['ReceiverCity'].map(cityDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,424.0,424.0,metro general,christchurch,christchurch,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,49.0,453.0,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,424.0,9.0,economy,christchurch,queenstown,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,49.0,453.0,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,49.0,220.0,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,49.0,49.0,metro general,auckland,greenlane,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,49.0,49.0,metro general,auckland,auckland,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,49.0,220.0,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,49.0,50.0,economy,auckland,whangarei,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,191.0,187.0,bop - waikato,auckland,rotorua,pu/lh/dl,20.60,stl linehaul


In [ ]:
Legdata['from_leg']=Legdata['from_leg'].map(cityDict)
Legdata['to_leg']=Legdata['to_leg'].map(cityDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,424.0,424.0,metro general,424.0,424.0,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,49.0,453.0,supereconomy,49.0,453.0,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,424.0,9.0,economy,424.0,80.0,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,49.0,453.0,supereconomy,49.0,453.0,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,49.0,220.0,supereconomy,49.0,220.0,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,49.0,49.0,metro general,49.0,NaN,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,49.0,49.0,metro general,49.0,49.0,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,49.0,220.0,supereconomy,49.0,220.0,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,49.0,50.0,economy,49.0,50.0,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,191.0,187.0,bop - waikato,49.0,187.0,pu/lh/dl,20.60,stl linehaul


#### **ServiceType**

In [ ]:
file = open("/content/drive/MyDrive/datasetW4/serviceType.csv", "r")
data2 = list(csv.reader(file, delimiter=","))
file.close()
data2

In [ ]:
serviceList = list()
for i in data2:
  for j in i:
    #print(j)
    serviceList.append(str(j).lower())
serviceList

In [ ]:
serviceDict = {}
serviceDict = {serviceList[i]: int(i+1) for i in range(len(serviceList))}
serviceDict

{'express-air': 1,
 'economy': 2,
 'point to point': 3,
 'container': 4,
 'metro general': 5,
 'express - road': 6,
 'chilled': 7,
 'supereconomy': 8,
 'frozen': 9,
 'hiab / crane truck': 10,
 'metro chilled': 11,
 'metro frozen': 12,
 'courier': 13,
 'full truck load': 14,
 'storage': 15,
 'noa - north': 16,
 'furniture': 17,
 'vehicle transport': 18,
 'rtd - taupo': 19,
 'bop - waikato': 20}

In [ ]:
with open('serviceDict.pkl', 'wb') as f:
    pickle.dump(serviceDict, f)
        
with open('serviceDict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
Legdata['ServiceType']=Legdata['ServiceType'].map(serviceDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,424.0,424.0,5,424.0,424.0,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,424.0,9.0,2,424.0,80.0,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,49.0,49.0,5,49.0,NaN,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,49.0,49.0,5,49.0,49.0,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,49.0,50.0,2,49.0,50.0,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,191.0,187.0,20,49.0,187.0,pu/lh/dl,20.60,stl linehaul


####**Customer Account name**

In [ ]:
file = open("/content/drive/MyDrive/datasetW4/CustomerAccount_name.csv", "r")
data3 = list(csv.reader(file, delimiter=","))
file.close()
data3

In [ ]:
accountList = list()
for i in data3[1:]:
  for j in i:
    #print(j)
    accountList.append(str(j).lower())
accountList

In [ ]:
accountDict = {}
accountDict = {accountList[i]: int(i+1) for i in range(len(accountList))}
accountDict

In [ ]:
with open('accountDict.pkl', 'wb') as f:
    pickle.dump(accountDict, f)
        
with open('accountDict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
Legdata['Account_name']=Legdata['Account_name'].map(accountDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,102.0,424.0,424.0,5,424.0,424.0,pu/dl,13.40,lk transport store rite logistics
2,132.0,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,stl linehaul
3,153.0,424.0,9.0,2,424.0,80.0,pu/lh/dl,30.00,booth's transport (si) ltd
4,132.0,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,stl linehaul
6,132.0,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,stl linehaul
7,132.0,49.0,49.0,5,49.0,NaN,pu/lh/dl,19.56,cargo plus ltd
8,132.0,49.0,49.0,5,49.0,49.0,pu/lh/dl,18.90,cargo plus ltd
9,132.0,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,stl linehaul
10,93.0,49.0,50.0,2,49.0,50.0,pu/lh/dl,131.26,mainstream nz ltd
11,132.0,191.0,187.0,20,49.0,187.0,pu/lh/dl,20.60,stl linehaul


#### **Carrier name**

In [ ]:
file = open("/content/drive/MyDrive/datasetW4/CarrierAccount_name.csv", "r")
data4 = list(csv.reader(file, delimiter=","))
file.close()
data4

In [ ]:
carrierList = list()
for i in data4[1:]:
  for j in i:
    #print(j)
    carrierList.append(str(j).lower())
carrierList

In [ ]:
carrierDict = {}
carrierDict = {carrierList[i]: int(i+1) for i in range(len(carrierList))}
carrierDict

{'lakeland contracting': 1,
 'cv compton ltd': 2,
 'auckland crane': 3,
 'booths transport ltd': 4,
 'tomoana warehousing ltd': 5,
 'express movers ': 6,
 'lt transport limited': 7,
 'gisborne hiabs ltd': 8,
 'buxton moves': 9,
 'bascik avenger cresent': 10,
 'move logistics ltd': 11,
 'hilton haulage ltd partnership': 12,
 'onsend ltd': 13,
 'n j woods ltd': 14,
 'dunamis transport ltd': 15,
 'collins lifting ltd': 16,
 'dave hoskin carriers ltd': 17,
 'ian roebuck crane hire limited': 18,
 'on time eddy': 19,
 'les harrison transport ltd': 20,
 'auckland cranes ltd': 21,
 'woofe contracting ltd': 22,
 'timatanga enterprises ltd': 23,
 'r&r hiab': 24,
 'wills contracting': 25,
 'lk transport tga': 26,
 'tappers ltd': 27,
 'hiab crane services ltd': 28,
 'brosnan transport ltd': 29,
 'akaroa frieght ltd': 30,
 'transfreight': 31,
 'l & a cotton heavy haulage': 32,
 'mainstream nz ltd': 33,
 "booth's transport (si) ltd": 34,
 'conroys removals ltd': 35,
 'whitfield transport': 36,
 'fre

In [ ]:
Legdata['Carrier_name']=Legdata['Carrier_name'].map(carrierDict)
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,102.0,424.0,424.0,5,424.0,424.0,pu/dl,13.40,86
2,132.0,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,95
3,153.0,424.0,9.0,2,424.0,80.0,pu/lh/dl,30.00,34
4,132.0,49.0,453.0,8,49.0,453.0,pu/lh/dl,47.16,95
6,132.0,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,95
7,132.0,49.0,49.0,5,49.0,NaN,pu/lh/dl,19.56,55
8,132.0,49.0,49.0,5,49.0,49.0,pu/lh/dl,18.90,55
9,132.0,49.0,220.0,8,49.0,220.0,pu/lh/dl,48.57,95
10,93.0,49.0,50.0,2,49.0,50.0,pu/lh/dl,131.26,33
11,132.0,191.0,187.0,20,49.0,187.0,pu/lh/dl,20.60,95


In [ ]:
Legdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19899 entries, 1 to 23647
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  19898 non-null  float64
 1   SenderCity    19896 non-null  float64
 2   ReceiverCity  19850 non-null  float64
 3   ServiceType   19899 non-null  int64  
 4   from_leg      18212 non-null  float64
 5   to_leg        17146 non-null  float64
 6   leg_type      19899 non-null  object 
 7   LeggingCost   19899 non-null  float64
 8   Carrier_name  19899 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 1.5+ MB


#### **Fill null values**

In [ ]:
print(Legdata['Account_name'].isnull().sum())
print(Legdata['SenderCity'].isnull().sum())
print(Legdata['ReceiverCity'].isnull().sum())
print(Legdata['ServiceType'].isnull().sum())
print(Legdata['Carrier_name'].isnull().sum())

1
3
49
0
0


In [ ]:
Legdata[Legdata['ReceiverCity'].isnull()]

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
431,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,19.56,55
623,132.0,49.0,NaN,5,49.0,49.0,pu/dl,19.56,55
5299,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,45.36,55
5908,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,25.04,55
7192,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,19.56,55
7309,132.0,49.0,NaN,5,49.0,NaN,pu/lh/dl,18.90,55
7492,132.0,49.0,NaN,5,49.0,NaN,pu/lh/dl,75.60,55
7916,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,23.28,55
8021,132.0,49.0,NaN,8,49.0,NaN,pu/lh/dl,121.11,95
8300,132.0,49.0,NaN,5,49.0,49.0,pu/lh/dl,19.56,55


In [ ]:
indices = [431,623,5299,5908,7192,7309,7492,7916,8300,8594,9284,9342,9736,10194,10841,11431,11575,11915,12448,12777,12829,12837,13646,13832,13841,14300,15090,15214,16683,17105,17179,17473,17549,19180,19530,
           19838,20445,20560,20742,23146,23432]
Legdata.loc[indices,'ReceiverCity'] = 49.0

print(Legdata.head(16))

In [ ]:
Ldata[Legdata['ReceiverCity'].isnull()]

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
17390,belgotex nz ltd,auckland,masterton,supereconomy,auckland,masterton,pu/lh/dl,47.16,stl linehaul


In [ ]:
indices = [8021,9049,11318,11440,11726,19935,20522]
Legdata.loc[indices,'ReceiverCity'] = 453.0

In [ ]:
indices = [17390]
Legdata.loc[indices,'ReceiverCity'] = 203.0

In [ ]:
Ldata[Legdata['SenderCity'].isnull()]

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
2623,belgotex nz ltd,auckland,auckland,metro general,auckland,auckland,pu/dl,18.90,cargo plus ltd
11190,belgotex nz ltd,auckland,auckland,metro general,auckland,auckland,pu/dl,22.12,cargo plus ltd
14854,belgotex nz ltd,auckland,auckland,metro general,auckland,auckland,pu/dl,33.45,cargo plus ltd


In [ ]:
indices = [2623,11190,14854]
Legdata.loc[indices,'SenderCity'] = 49.0

In [ ]:
Ldata[Legdata['Account_name'].isnull()]

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
23505,freightplus ltd,christchurch,christchurch,metro general,wigram,yaldhurst,pu/lh/dl,16.08,lk transport store rite logistics


In [ ]:
indices = [23505]
Legdata.loc[indices,'Account_name'] = 59.0

In [ ]:
Legdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19899 entries, 1 to 23647
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  19899 non-null  float64
 1   SenderCity    19899 non-null  float64
 2   ReceiverCity  19899 non-null  float64
 3   ServiceType   19899 non-null  int64  
 4   from_leg      18212 non-null  float64
 5   to_leg        17146 non-null  float64
 6   leg_type      19899 non-null  object 
 7   LeggingCost   19899 non-null  float64
 8   Carrier_name  19899 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 2.0+ MB


In [ ]:
Legdata.isnull().sum()

Account_name       0
SenderCity         0
ReceiverCity       0
ServiceType        0
from_leg        1687
to_leg          2753
leg_type           0
LeggingCost        0
Carrier_name       0
dtype: int64

In [ ]:
Legdata2 = Legdata.copy()

In [ ]:
df_preprocessed=Legdata.copy()
df_preprocessed.to_csv('LeggingData.csv', index=False)

In [ ]:
Legdata=Legdata.drop(['from_leg'], axis=1)
Legdata=Legdata.drop(['to_leg'], axis=1)
Legdata=Legdata.drop(['leg_type'], axis=1)


In [ ]:
Legdata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,LeggingCost,Carrier_name
1,102.0,424.0,424.0,5,13.40,86
2,132.0,49.0,453.0,8,47.16,95
3,153.0,424.0,9.0,2,30.00,34
4,132.0,49.0,453.0,8,47.16,95
6,132.0,49.0,220.0,8,48.57,95
7,132.0,49.0,49.0,5,19.56,55
8,132.0,49.0,49.0,5,18.90,55
9,132.0,49.0,220.0,8,48.57,95
10,93.0,49.0,50.0,2,131.26,33
11,132.0,191.0,187.0,20,20.60,95


In [ ]:
Legdata.isnull().sum()

Account_name    0
SenderCity      0
ReceiverCity    0
ServiceType     0
LeggingCost     0
Carrier_name    0
dtype: int64

In [ ]:
df_preprocessed=Legdata.copy()
df_preprocessed.to_csv('LeggingDataCleaned2.csv', index=False)

In [ ]:
cLegData=pd.read_csv("/content/drive/MyDrive/datasetW4/LeggingDataCleaned.csv")
cLegData.head(4)

,Account_name,SenderCity,ReceiverCity,ServiceType,LeggingCost,Carrier_name
0,102.0,424.0,424.0,5,13.40,86
1,132.0,49.0,453.0,8,47.16,95
2,153.0,424.0,9.0,2,30.00,34
3,132.0,49.0,453.0,8,47.16,95


In [ ]:
cLegData=pd.read_csv("/content/drive/MyDrive/datasetW4/LeggingDataCleaned2.csv")
cLegData.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,LeggingCost,Carrier_name
0,102.0,424.0,424.0,5,13.40,lk transport store rite logistics
1,132.0,49.0,453.0,8,47.16,stl linehaul
2,153.0,424.0,9.0,2,30.00,booth's transport (si) ltd
3,132.0,49.0,453.0,8,47.16,stl linehaul
4,132.0,49.0,220.0,8,48.57,stl linehaul
5,132.0,49.0,49.0,5,19.56,cargo plus ltd
6,132.0,49.0,49.0,5,18.90,cargo plus ltd
7,132.0,49.0,220.0,8,48.57,stl linehaul
8,93.0,49.0,50.0,2,131.26,mainstream nz ltd
9,132.0,191.0,187.0,20,20.60,stl linehaul


### **Duplicating data**

In [ ]:
cLegData.drop_duplicates()

In [ ]:
cLegData[cLegData[['Account_name',	'SenderCity',	'ReceiverCity',	'ServiceType',	'LeggingCost']].duplicated() == True]

In [ ]:
pd.set_option('display.max_rows', None)
cLegData['Carrier_name'].value_counts()

stl linehaul                          6919
cargo plus ltd                        3745
booth's transport (si) ltd            2157
bascik transport                      2084
mainstream nz ltd                     1639
t&g transport                          830
freightplus ltd                        721
lk transport store rite logistics      642
halls refrigeration                    183
transfreight                           160
linfox                                 137
angel transport                        109
pass the parcel                         70
combined freightlines                   57
urgent couriers ltd                     50
qantas couriers                         45
metro urgent                            39
jets transport ltd                      37
mackenzie transport ltd                 27
adept logistics ltd                     22
sub60                                   21
sealink logistics                       21
transcon                                21
sollys     

In [ ]:
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'smith transport']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'gisborne hiabs ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'conroys removals ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'auckland cranes ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'todd mcphee crane hire & transport']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'primelift hiab hire ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'wills contracting']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'hiab crane services ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'til freighting ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'n j woods ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'great barrier cartage ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'woofe contracting ltd']] * 10, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'on time eddy']] * 6, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'marlborough moving & storage co']] * 6, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'tappers ltd']] * 6, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'les harrison transport ltd']] * 6, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'grace removals']] * 6, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'express movers']] * 6, ignore_index=True)

In [ ]:
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'ian roebuck crane hire limited']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'move logistics ltd']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'fr8base']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'man with a van service']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'express movers']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'lift n shift ltd']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'tomoana warehousing ltd']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'timatanga enterprises ltd']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'croft combined carriers ltd']] * 5, ignore_index=True)
cLegData=cLegData.append([cLegData[cLegData['Carrier_name'] == 'raglan carriers']] * 5, ignore_index=True)

In [ ]:
cLegData[cLegData['Carrier_name'] == 'n j woods ltd']


,Account_name,SenderCity,ReceiverCity,ServiceType,LeggingCost,Carrier_name
16522,118.0,49.0,49.0,3,2850.0,n j woods ltd


In [ ]:
cLegData['Carrier_name'].value_counts()

stl linehaul                          6919
cargo plus ltd                        3745
booth's transport (si) ltd            2157
bascik transport                      2084
mainstream nz ltd                     1639
t&g transport                          830
freightplus ltd                        721
lk transport store rite logistics      642
halls refrigeration                    183
transfreight                           160
linfox                                 137
angel transport                        109
pass the parcel                         70
combined freightlines                   57
urgent couriers ltd                     50
qantas couriers                         45
metro urgent                            39
jets transport ltd                      37
raglan carriers                         30
mackenzie transport ltd                 27
croft combined carriers ltd             24
timatanga enterprises ltd               24
adept logistics ltd                     22
sealink log

In [ ]:
cLegData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20191 entries, 0 to 20190
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  20191 non-null  float64
 1   SenderCity    20191 non-null  float64
 2   ReceiverCity  20191 non-null  float64
 3   ServiceType   20191 non-null  int64  
 4   LeggingCost   20191 non-null  float64
 5   Carrier_name  20191 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 946.6+ KB


In [ ]:
df_preprocessed=cLegData.copy()
df_preprocessed.to_csv('LeggingDuplicated.csv', index=False)

For Deep learning only mapping Carrier names to numerical values

In [ ]:
cLegData2 = cLegData.copy()

In [ ]:
cLegData2['Carrier_name']=cLegData2['Carrier_name'].map(carrierDict)
cLegData2.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,LeggingCost,Carrier_name
0,102.0,424.0,424.0,5,13.40,86
1,132.0,49.0,453.0,8,47.16,95
2,153.0,424.0,9.0,2,30.00,34
3,132.0,49.0,453.0,8,47.16,95
4,132.0,49.0,220.0,8,48.57,95
5,132.0,49.0,49.0,5,19.56,55
6,132.0,49.0,49.0,5,18.90,55
7,132.0,49.0,220.0,8,48.57,95
8,93.0,49.0,50.0,2,131.26,33
9,132.0,191.0,187.0,20,20.60,95


In [ ]:
cLegData2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20191 entries, 0 to 20190
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  20191 non-null  float64
 1   SenderCity    20191 non-null  float64
 2   ReceiverCity  20191 non-null  float64
 3   ServiceType   20191 non-null  int64  
 4   LeggingCost   20191 non-null  float64
 5   Carrier_name  20191 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 946.6 KB


In [ ]:
#cLegData=cLegData.drop(['LeggingCost'], axis=1)
#cLegData=pd.read_csv("/content/drive/MyDrive/datasetW4/LeggingDataCleaned2.csv")
#cLegData.head(10)

## **Experiments (Train Test)**

**Catboost with weights**

In [ ]:
cLegData=pd.read_csv("/content/drive/MyDrive/datasetW4/LeggingDuplicated.csv")
cLegData.head(10)

In [ ]:
X=cLegData.iloc[:,:-1]
y=cLegData['Carrier_name']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(cLegData.iloc[:,:-1],
    cLegData['Carrier_name'],
    test_size=0.3,
    random_state=0)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

#clf = CatBoostClassifier(verbose=0, n_estimators=100 , class_weights=class_weights)
#clf.fit(X_train,y_train)
clf = CatBoostClassifier(loss_function='MultiClass', class_weights=class_weights)
clf.fit(X_train, y_train)

Learning rate set to 0.09057
0:	learn: 3.4845051	total: 280ms	remaining: 4m 39s
1:	learn: 2.9561125	total: 459ms	remaining: 3m 49s
2:	learn: 2.6944336	total: 638ms	remaining: 3m 32s
3:	learn: 2.4434631	total: 811ms	remaining: 3m 21s
4:	learn: 2.2642534	total: 970ms	remaining: 3m 12s
5:	learn: 2.1434534	total: 1.12s	remaining: 3m 6s
6:	learn: 2.0244335	total: 1.29s	remaining: 3m 3s
7:	learn: 1.8940805	total: 1.44s	remaining: 2m 58s
8:	learn: 1.7989795	total: 1.6s	remaining: 2m 56s
9:	learn: 1.7061821	total: 1.75s	remaining: 2m 53s
10:	learn: 1.6291770	total: 1.91s	remaining: 2m 51s
11:	learn: 1.5507444	total: 2.07s	remaining: 2m 50s
12:	learn: 1.4826703	total: 2.23s	remaining: 2m 49s
13:	learn: 1.4232300	total: 2.4s	remaining: 2m 48s
14:	learn: 1.3579120	total: 2.55s	remaining: 2m 47s
15:	learn: 1.3057386	total: 2.7s	remaining: 2m 46s
16:	learn: 1.2566484	total: 2.86s	remaining: 2m 45s
17:	learn: 1.2160084	total: 3.01s	remaining: 2m 44s
18:	learn: 1.1759692	total: 3.16s	remaining: 2m 43

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
result2 = accuracy_score(y_test,y_pred)
print('Accuracy:',result2*100,'%')

np.set_printoptions(threshold=200)
result = confusion_matrix(y_test, y_pred)
print('\nConfusion Matrix:')
print(result)

#result1 = classification_report(y_test, ypred)
#print('Classification Report:',)
#print (result1)

Accuracy: 75.0412677451304 %

Confusion Matrix:
[[ 4  0  0 ...  0  0  0]
 [ 0 23  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  3  0]
 [ 0  0  0 ...  0  0  3]]


**Catboost**

In [ ]:
clf = CatBoostClassifier(verbose=0, n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
result2 = accuracy_score(y_test,y_pred)
print('Accuracy:',result2*100,'%')

np.set_printoptions(threshold=200)
result = confusion_matrix(y_test, y_pred)
print('\nConfusion Matrix:')
print(result)

Accuracy: 83.90557939914163 %

Confusion Matrix:
[[ 1  0  0 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  3  0]
 [ 0  0  0 ...  0  0  3]]


**XGBoost**

In [ ]:
#XGBOOST
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

result2 = accuracy_score(y_test,y_pred)
print('Accuracy:',result2*100,'%')
print(confusion_matrix(y_test, y_pred))

Accuracy: 86.9098712446352 %
[[ 0  0  0 ...  0  0  0]
 [ 0 24  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  3  0]
 [ 0  0  0 ...  0  0  3]]


**Light GBM**

In [ ]:
#LightGBM
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='multiclass', random_state=5)
lgbm.fit(X_train, y_train)

y_pred = lgbm.predict(X_test)

result2 = accuracy_score(y_test,y_pred)
print('Accuracy:',result2*100,'%')
print(confusion_matrix(y_test, y_pred))

Accuracy: 50.08253549026082 %
[[ 0  0  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


**Naive Bayes**

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import BernoulliNB
#from sklearn.model_selection import train_test_split

bnb = BernoulliNB(binarize=0.0)
bnb.fit(X_train, y_train)
bnb.score(X_test, y_test)

0.3403763618355893

**DecisionTree**

In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.7160779135028063


**KNN**

In [ ]:
from sklearn.model_selection import GridSearchCV

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
best_model = clf.fit(X_train,y_train)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 2 members, which is less than n_splits=10.



Best leaf_size: 7
Best p: 1
Best n_neighbors: 1


In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier


#Create KNN Classifier
knn = KNeighborsClassifier(leaf_size=7, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8075272367117861


In [ ]:
predicted= knn.predict([[132.0,	49.0,	49.0,	10,	48.57	]]) # 0:Overcast, 2:Mild
print(predicted)

['cargo plus ltd']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[ 4  0  0 ...  0  0  0]
 [ 0 13  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  3  0]
 [ 0  0  0 ...  0  0  3]]
                                    precision    recall  f1-score   support

               adept logistics ltd       0.29      0.40      0.33        10
                   angel transport       0.19      0.43      0.26        30
               auckland cranes ltd       1.00      1.00      1.00         5
                  bascik transport       0.63      0.78      0.70       618
        booth's transport (si) ltd       0.87      0.86      0.86       667
              booths transport ltd       0.00      0.00      0.00         3
                    cargo plus ltd       0.93      0.98      0.95      1116
             combined freightlines       0.33      0.12      0.17        17
            combined logistics ltd       0.00      0.00      0.00         2
              conroys removals ltd       0.80      1.00      0.89         4
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
# save the model to disk
import pickle
filename = 'KNN_model.sav'
pickle.dump(knn, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = metrics.accuracy_score(y_test, y_pred)
print(result)

0.8075272367117861


**Test a sample with Trained model**

In [ ]:
Ldata.head(10)

,Account_name,SenderCity,ReceiverCity,ServiceType,from_leg,to_leg,leg_type,LeggingCost,Carrier_name
1,auslink logistics limited,christchurch,christchurch,metro general,christchurch,christchurch,pu/dl,13.40,lk transport store rite logistics
2,belgotex nz ltd,auckland,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
3,dhl express - chch,christchurch,lake hayes,economy,christchurch,queenstown,pu/lh/dl,30.00,booth's transport (si) ltd
4,belgotex nz ltd,auckland,wellington,supereconomy,auckland,wellington,pu/lh/dl,47.16,stl linehaul
6,belgotex nz ltd,auckland,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
7,belgotex nz ltd,auckland,auckland,metro general,auckland,greenlane,pu/lh/dl,19.56,cargo plus ltd
8,belgotex nz ltd,auckland,auckland,metro general,auckland,auckland,pu/lh/dl,18.90,cargo plus ltd
9,belgotex nz ltd,auckland,new plymouth,supereconomy,auckland,new plymouth,pu/lh/dl,48.57,stl linehaul
10,pyrotek products ltd,auckland,whangarei,economy,auckland,whangarei,pu/lh/dl,131.26,mainstream nz ltd
11,belgotex nz ltd,tauranga,rotorua,bop - waikato,auckland,rotorua,pu/lh/dl,20.60,stl linehaul


In [ ]:
serviceDict

{'express-air': 1,
 'economy': 2,
 'point to point': 3,
 'container': 4,
 'metro general': 5,
 'express - road': 6,
 'chilled': 7,
 'supereconomy': 8,
 'frozen': 9,
 'hiab / crane truck': 10,
 'metro chilled': 11,
 'metro frozen': 12,
 'courier': 13,
 'full truck load': 14,
 'storage': 15,
 'noa - north': 16,
 'furniture': 17,
 'vehicle transport': 18,
 'rtd - taupo': 19,
 'bop - waikato': 20}

In [ ]:
loaded_model = pickle.load(open('/content/drive/MyDrive/datasetW4/KNN_model.sav', 'rb'))

accountName = 'belgotex nz ltd'
senderCity = 'auckland'
receiverCity = 'christchurch'
serviceType = 'supereconomy'
leggingCost = 48.57

accountName = accountDict[accountName]
senderCity = cityDict[senderCity]
receiverCity = cityDict[receiverCity]
serviceType = serviceDict[serviceType]
print("accountName:",accountName,"\nsenderCity:",senderCity,"\nreceiverCity:",receiverCity,"\nserviceType:",serviceType,"\nleggingCost:", leggingCost)

predicted= loaded_model.predict([[accountName, senderCity, receiverCity, serviceType, leggingCost]])
print("\nRecommending :", predicted[0].upper())

accountName: 132 
senderCity: 49 
receiverCity: 424 
serviceType: 8 
leggingCost: 48.57

Recommending : STL LINEHAUL


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



In [ ]:
pickle.format_version

'4.0'

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
sklearn. __version__


'1.0.2'

In [ ]:
sns. __version__

'0.11.2'

In [ ]:
loaded_model = pickle.load(open('/content/KNN_model.sav', 'rb'))
predicted= loaded_model.predict([[102.0,	19.0,	49.0,	10,	48.57	]]) # 0:Overcast, 2:Mild
print(predicted)

["booth's transport (si) ltd"]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_train, y_pred)
rmse = sqrt(mse)
rmse

**KNN - 2**

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5, weights = 'uniform',algorithm = 'brute',metric = 'minkowski')
knn.fit(X_train, y_train)
y_hat = knn.predict(X_train)
y_knn = knn.predict(X_test)
print('Training set accuracy: ', metrics.accuracy_score(y_train, y_hat))
print('Test set accuracy: ',metrics.accuracy_score(y_test, y_knn))

Training set accuracy:  0.8394537607019034
Test set accuracy:  0.7812809508088479


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_knn))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_knn))

[[1 0 0 ... 0 0 0]
 [0 4 2 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 3]]
                                    precision    recall  f1-score   support

               adept logistics ltd       0.33      0.10      0.15        10
                   angel transport       0.09      0.13      0.11        30
               auckland cranes ltd       0.62      1.00      0.77         5
                  bascik transport       0.63      0.70      0.66       618
        booth's transport (si) ltd       0.86      0.85      0.85       667
              booths transport ltd       0.00      0.00      0.00         3
                    cargo plus ltd       0.92      0.97      0.95      1116
             combined freightlines       0.43      0.18      0.25        17
            combined logistics ltd       0.00      0.00      0.00         2
              conroys removals ltd       0.80      1.00      0.89         4
       croft combined carriers ltd       0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(knn, X, y, cv =5)
print('Model accuracy: ',np.mean(scores))

**KNN - 3**

In [ ]:
0.7814


**Random Forest**

In [ ]:
cLegData2=pd.read_csv("/content/drive/MyDrive/datasetW4/LeggingDataCleaned.csv")
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(cLegData2.iloc[:,:-1],
    cLegData2['Carrier_name'],
    test_size=0.3,
    random_state=0)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

#### **LSTM**

In [ ]:
X=X_train
y=y_train
X=np.expand_dims(X,-1)
print(X.shape)
print(y.shape)

(14133, 5, 1)
(14133,)


In [ ]:
y=np.array(y)
y=y.reshape(-1, 1)
y.shape

(14133, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y) 
y=y.toarray()

In [ ]:
#preprocessing test data
X_test=np.expand_dims(X_test,-1)

ytest=np.array(y_test)
ytest=ytest.reshape(-1, 1)
enc = OneHotEncoder()
ytest = enc.fit_transform(ytest) 
ytest=ytest.toarray()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential([
      LSTM(512,return_sequences=False, input_shape=(5,1)),
      Dense(256, activation='relu'),
      Dropout(0.2),
      Dense(128, activation='relu'),
      Dropout(0.2),
      Dense(62, activation='softmax'),
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 512)               1052672   
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 62)                7998      
                                                                 
Total params: 1,224,894
Trainable params: 1,224,894
No

In [ ]:
history= model.fit(X, y, validation_split=0.2, epochs=100, batch_size=512, shuffle=True)

Epoch 1/100
23/23 [==============================] - 13s 448ms/step - loss: 2.4500 - accuracy: 0.3960 - val_loss: 1.6339 - val_accuracy: 0.5497
Epoch 2/100
23/23 [==============================] - 10s 422ms/step - loss: 1.4650 - accuracy: 0.5946 - val_loss: 1.2345 - val_accuracy: 0.6753
Epoch 3/100
23/23 [==============================] - 10s 420ms/step - loss: 1.1936 - accuracy: 0.6941 - val_loss: 1.1048 - val_accuracy: 0.7237
Epoch 4/100
23/23 [==============================] - 10s 423ms/step - loss: 1.0909 - accuracy: 0.7252 - val_loss: 1.0638 - val_accuracy: 0.7283
Epoch 5/100
23/23 [==============================] - 10s 422ms/step - loss: 1.0308 - accuracy: 0.7395 - val_loss: 1.0035 - val_accuracy: 0.7503
Epoch 6/100
23/23 [==============================] - 11s 461ms/step - loss: 0.9905 - accuracy: 0.7462 - val_loss: 0.9743 - val_accuracy: 0.7520
Epoch 7/100
23/23 [==============================] - 10s 426ms/step - loss: 0.9509 - accuracy: 0.7550 - val_loss: 0.9364 - val_accuracy:

In [ ]:
scores = model.evaluate(X_test, ytest)
LSTM_accuracy = scores[1]*100
print('Test accuracy: ', scores[1]*100, '%')

190/190 [==============================] - 3s 17ms/step - loss: 2.9183 - accuracy: 0.7304
Test accuracy:  73.04390668869019 %


In [ ]:
# save the model to disk
import pickle
filename = 'LSTM_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.evaluate(X_test,ytest)
print(result)

190/190 [==============================] - 4s 21ms/step - loss: 2.9183 - accuracy: 0.7304
[2.918288230895996, 0.7304390668869019]


####**Sampled**

In [ ]:
stl=  cLegData.loc[cLegData['Carrier_name'] =='stl linehaul']
cargo=  cLegData.loc[cLegData['Carrier_name'] =='cargo plus ltd']
booth=  cLegData.loc[cLegData['Carrier_name'] =="booth's transport (si) ltd"]
bascik=  cLegData.loc[cLegData['Carrier_name'] =='bascik transport']
mainstream=  cLegData.loc[cLegData['Carrier_name'] =='mainstream nz ltd']
tng=  cLegData.loc[cLegData['Carrier_name'] =='t&g transport']
freightplus=  cLegData.loc[cLegData['Carrier_name'] =='freightplus ltd']
lk=  cLegData.loc[cLegData['Carrier_name'] =='lk transport store rite logistics']
halls=  cLegData.loc[cLegData['Carrier_name'] =='halls refrigeration']
transfreight=  cLegData.loc[cLegData['Carrier_name'] =='transfreight']
linfox=  cLegData.loc[cLegData['Carrier_name'] =='linfox']
angel=  cLegData.loc[cLegData['Carrier_name'] =='angel transport']
passt=  cLegData.loc[cLegData['Carrier_name'] =='pass the parcel']
combined=  cLegData.loc[cLegData['Carrier_name'] =='combined freightlines']
urgent=  cLegData.loc[cLegData['Carrier_name'] =='urgent couriers ltd']
qantas=  cLegData.loc[cLegData['Carrier_name'] =='qantas couriers']
metro=  cLegData.loc[cLegData['Carrier_name'] =='metro urgent']
jets=  cLegData.loc[cLegData['Carrier_name'] =='jets transport ltd']
mackenzie=  cLegData.loc[cLegData['Carrier_name'] =='mackenzie transport ltd']

SampData = pd.concat([stl,cargo,booth,bascik,mainstream,tng,freightplus,lk,halls,transfreight,linfox,angel,passt,combined,urgent,qantas,metro,jets,mackenzie], axis = 0)

In [ ]:
SampData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19651 entries, 1 to 19628
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  19651 non-null  float64
 1   SenderCity    19651 non-null  float64
 2   ReceiverCity  19651 non-null  float64
 3   ServiceType   19651 non-null  int64  
 4   LeggingCost   19651 non-null  float64
 5   Carrier_name  19651 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 1.0+ MB


## **Oversampling**

In [ ]:
X=SampData.iloc[:,:-1]
y=SampData['Carrier_name']

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(sampling_strategy = {'mackenzie transport ltd': 1000, 't&g transport':1000,
                     'freightplus ltd':1000,'lk transport store rite logistics':1000,'halls refrigeration':1000,'transfreight':1000,'linfox':1000,'angel transport':1000,'pass the parcel':1000,'combined freightlines':1000,
										           'urgent couriers ltd':1000,'qantas couriers':1000,'metro urgent':1000,'jets transport ltd':1000})

X,y=oversample.fit_resample(X,y)

undersample = RandomUnderSampler()
X,y=undersample.fit_resample(X,y)

counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]


Counter({'angel transport': 1000, 'bascik transport': 1000, "booth's transport (si) ltd": 1000, 'cargo plus ltd': 1000, 'combined freightlines': 1000, 'freightplus ltd': 1000, 'halls refrigeration': 1000, 'jets transport ltd': 1000, 'linfox': 1000, 'lk transport store rite logistics': 1000, 'mackenzie transport ltd': 1000, 'mainstream nz ltd': 1000, 'metro urgent': 1000, 'pass the parcel': 1000, 'qantas couriers': 1000, 'stl linehaul': 1000, 't&g transport': 1000, 'transfreight': 1000, 'urgent couriers ltd': 1000})


In [ ]:
y.value_counts()

angel transport                      1000
mackenzie transport ltd              1000
transfreight                         1000
t&g transport                        1000
stl linehaul                         1000
qantas couriers                      1000
pass the parcel                      1000
metro urgent                         1000
mainstream nz ltd                    1000
lk transport store rite logistics    1000
bascik transport                     1000
linfox                               1000
jets transport ltd                   1000
halls refrigeration                  1000
freightplus ltd                      1000
combined freightlines                1000
cargo plus ltd                       1000
booth's transport (si) ltd           1000
urgent couriers ltd                  1000
Name: Carrier_name, dtype: int64

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19000 entries, 0 to 18999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Account_name  19000 non-null  float64
 1   SenderCity    19000 non-null  float64
 2   ReceiverCity  19000 non-null  float64
 3   ServiceType   19000 non-null  int64  
 4   LeggingCost   19000 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 742.3 KB


In [ ]:
print(X.shape)
print(y.shape)

(19000, 5)
(19000,)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

from sklearn.model_selection import GridSearchCV

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
best_model = clf.fit(X_train,y_train)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 47
Best p: 1
Best n_neighbors: 1


In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier


#Create KNN Classifier
knn = KNeighborsClassifier(leaf_size=47, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=1,
                     weights='uniform')

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8575438596491228


In [ ]:
predicted= knn.predict([[132.0,	49.0,	49.0,	10,	48.57	]]) # 0:Overcast, 2:Mild
print(predicted)

['cargo plus ltd']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[258   3   0   0   0  12   1   0   0   0   1   4   0   0   0   0   0   0
    0]
 [  1 177   6   0   0  13   2   0  11   0   2   9   6   2  18  23   0   7
   10]
 [  1   6 248   0   9   3   0   2   2   4   7   5   1   1   1   7   0   1
    1]
 [  4   0   0 262   0   7   0   0   0   0   0   1   1   0   0   0   0   0
    0]
 [  0   0   5   0 264   0   0   0   0  21   1   0   0   0   0   0   0   0
    0]
 [ 22  12   4  16   0 194   2   0   1   0   0  14   5   0   3   7   0  27
    4]
 [  0   3   0   0   0   0 280   0   0   0   0   0   0   0   3   1   1   0
    0]
 [  0   0   3   0   1   0   1 308   1   0   3   2   0   0   0   0   0   0
    0]
 [  1   5   1   0   1   0   0   0 303   0   0   2   0   0   1   2   2   0
    1]
 [  0   0   4   0   3   0   0   0   0 268   3   2   0   0   0   0   0   0
    0]
 [  0   0  13   0   0   0   0   0   0   0 293   1   0   0   1   1   0   0
    0]
 [  3  36  10   0   2  21   5   4   3   0   1 172   6   2   1  21   3  19
    4]
 [  1   3   0   3   0   5   

In [ ]:
# save the model to disk
import pickle
filename = 'KNN_ResampledModel.sav'
pickle.dump(knn, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = metrics.accuracy_score(y_test, y_pred)
print(result)

0.8575438596491228


In [ ]:
'stl linehaul': 1000, 'cargo plus ltd': 1000, "booth's transport (si) ltd": 1000,'bascik transport':1000,'mainstream nz ltd':1000,

In [ ]:

# Oversample with SMOTE and random undersample for imbalanced dataset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from matplotlib import pyplot
from numpy import where
# define dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
#	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# summarize class distribution
counter = Counter(y)
print(counter)
# define pipeline
over = SMOTE("minority")
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X, y = pipeline.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()